In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apagche.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,843 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,349 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricte

In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import year
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Show DataFrame
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
df = df.withColumn("year_sold", year(df['date']))
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|     2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|     2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|     2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|     2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|     2022|


In [7]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sales')

In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT year_sold, ROUND(AVG(price), 2) AS AvgPrice
FROM sales
WHERE bedrooms = 4
GROUP BY year_sold
"""
spark.sql(query).show()


+---------+---------+
|year_sold| AvgPrice|
+---------+---------+
|     2022|296363.88|
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
+---------+---------+



In [9]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

query = """
SELECT date_built, ROUND(AVG(price), 2) AS AvgPrice
FROM sales
WHERE bedrooms = 3
  AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(query).show()

+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [10]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
SELECT date_built, ROUND(AVG(price), 2) AS AvgPrice
FROM sales
WHERE bedrooms = 3
  AND bathrooms = 3
  AND floors = 2
  AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built
"""
spark.sql(query).show()

+----------+---------+
|date_built| AvgPrice|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [17]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
query = """
SELECT view, ROUND(AVG(price), 2) AS AvgPrice
FROM sales
GROUP BY view
HAVING AvgPrice >= 350000
ORDER BY view Desc
"""

spark.sql(query).show()
print("---Runtime %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  AvgPrice|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

---Runtime 0.5658371448516846 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("sales")

# Verify that the table has been cached
cached_tables = spark.catalog.listTables()
if "sales" in [table.name for table in cached_tables]:
    print("Table 'sales' has been cached.")
else:
    print("Table 'sales' could not be cached.")

Table 'sales' has been cached.


In [23]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [24]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
query = """
SELECT view, ROUND(AVG(price), 2) AS AvgPrice
FROM sales
GROUP BY view
HAVING AvgPrice >= 350000
ORDER BY view Desc
"""

spark.sql(query).show()
print("---Runtime with Cached Table %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  AvgPrice|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

---Runtime with Cached Table 0.33133363723754883 seconds ---


In [25]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.partitionBy("date_built").parquet("partitioned_home_sales.parquet", mode="overwrite")


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year_sold|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+---------+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|     2021|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|     2020|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|     2019|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|     2019|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|     2021|      2015|


In [ ]:
# 11. Read the parquet formatted data.
partitioned_df = spark.read.parquet("partitioned_home_sales.parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView("partitioned_sales")
partitioned_df.show()

In [27]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) AS AvgPrice
FROM partitioned_sales
GROUP BY view
HAVING AvgPrice >= 350000
ORDER BY view Desc
"""

result = spark.sql(query).show(5)
print("---Runtime for parquet %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|  AvgPrice|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
+----+----------+
only showing top 5 rows

---Runtime for parquet 0.753791093826294 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("sales")

In [29]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('sales')

False